In [1]:
!pip install POT
!pip install gudhi

In [2]:
%matplotlib inline

# Imports
import os
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pandas as pd
import gudhi as gd
import multiprocessing
import pickle
from pathlib import Path
from tqdm import tqdm
import itertools
!pip install scikit-learn==1.4.2
import gudhi.wasserstein as gw

## Processing Images

In [4]:
## Gray Scale channels

def process_grey_images(image_dir, processed_images):
    # List of image files (replace this with your actual list of files)
    image_files = os.listdir(image_dir)  # Assuming the image folder is correct
    
    # Define the path to save the grayscale images
    gray_folder = os.path.join(processed_images, "grayscale_channel")
    
    # Check if the grayscale folder exists; if not, create it
    if not os.path.exists(gray_folder):
        os.makedirs(gray_folder, exist_ok=True)
        
    # Initialize a list to keep track of successfully processed image file names
    processed_files = []
    
    # Loop through each image file in the list 'image_files'
    for img_name in image_files:
        # Create the full path to the image file
        img_path = os.path.join(image_dir, img_name)
        
        # Read the image using OpenCV
        img = cv2.imread(img_path)
        
        # If the image is not read correctly (e.g., file is corrupted or not found), skip it
        if img is None:
            print(f"Skipping invalid image: {img_name}")
            continue
        
        # Convert the image from RGB (color) to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # OpenCV loads images as BGR, not RGB
        
        # Save the grayscale image to the specified folder with the same file name
        cv2.imwrite(os.path.join(gray_folder, img_name), gray_img)
        
        # Add the image name to the list of processed files
        processed_files.append(img_name)
        
    # Print out how many images were successfully processed and saved
    print(f"Processed {len(processed_files)} images and saved them in grayscale folder.")


In [5]:
##  RGB channels

def process_RGB_images(image_dir, processed_images):
    # List of image files (replace this with your actual list of files)
    image_files = os.listdir(image_dir)  # Assuming the image folder is correct

    # Define folders to save individual RGB channel images
    red_folder = os.path.join(processed_images, "red_channel")
    green_folder = os.path.join(processed_images, "green_channel")
    blue_folder = os.path.join(processed_images, "blue_channel")

    # Create the folders
    os.makedirs(red_folder, exist_ok=True)
    os.makedirs(green_folder, exist_ok=True)
    os.makedirs(blue_folder, exist_ok=True)

    # Initialize a list to keep track of successfully processed image file names
    processed_files = []

    # Loop through each image file in the list 'image_files'
    for img_name in image_files:
        # Get the full path to the image
        img_path = os.path.join(image_dir, img_name)

        # Read the image using OpenCV (default color format is BGR)
        img = cv2.imread(img_path)

        # If image loading fails (e.g., corrupted or missing file), skip it
        if img is None:
            print(f"Skipping invalid image: {img_name}")
            continue

        # Convert the image from BGR to RGB format (since OpenCV loads in BGR by default)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Split the RGB image into individual R, G, and B channels
        R, G, B = cv2.split(img)

        # Save each channel as a separate image
        cv2.imwrite(os.path.join(red_folder, f"R_{img_name}"), R)
        cv2.imwrite(os.path.join(green_folder, f"G_{img_name}"), G)
        cv2.imwrite(os.path.join(blue_folder, f"B_{img_name}"), B)

        # Record the processed image name
        processed_files.append(img_name)

    # Print out how many images were successfully processed and saved
    print(f"Processed {len(processed_files)} images and saved them in RGB channel folders.")

In [6]:
## Edge channel

def process_edge_images(image_dir, processed_images):
    # List of image files (replace this with your actual list of files)
    image_files = os.listdir(image_dir)  # Assuming the image folder is correct

    # Define the folder for edge-detected images
    edge_folder = os.path.join(processed_images, "edge_channel")

    # Create the folder if it doesn't already exist
    if not os.path.exists(edge_folder):
        os.makedirs(edge_folder, exist_ok=True)

    # List to keep track of successfully processed image file names
    processed_files = []

    # Loop through each image in the provided list of image files
    for img_name in image_files:
        # Create the full file path for the current image
        img_path = os.path.join(image_dir, img_name)

        # Read the image in grayscale mode (since edge detection works on single-channel images)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # If the image couldn't be read (e.g., missing or corrupted), skip it
        if img is None:
            print(f"Skipping invalid image: {img_name}")
            continue

        # Apply Canny edge detection to the grayscale image
        edges = cv2.Canny(img, 100, 200)

        # Save the resulting edge-detected image to the output folder
        cv2.imwrite(os.path.join(edge_folder, img_name), edges)

        # Add the image name to the list of successfully processed images
        processed_files.append(img_name)

    # Print out the total number of images that were processed and saved
    print(f"Processed {len(processed_files)} images and saved them in the edge-detection folder.")


## Persistent Homology

In [8]:
def compute_persistent_homology(image_dir, persistence_results_dir):
    """
    Compute persistent homology for grayscale images located in subfolders of a given directory.
    Each subfolder is processed separately, and results are saved as pickle files.
    
    Args:
        image_dir (str): Path to the main directory containing subfolders of images.
        persistence_results_dir (str): Directory where persistence results will be saved.
    
    Returns:
        dict: A nested dictionary of persistence results {subfolder_name: {image_name: persistence_data}}.
    """
    
    # Get all subfolder paths inside the image directory
    subfolders = [
        os.path.join(image_dir, subfolder)
        for subfolder in os.listdir(image_dir)
        if os.path.isdir(os.path.join(image_dir, subfolder))
    ]

    # --- Helper function: Convert a grayscale image to a Cubical Complex ---
    def image_to_cubical_complex(image_gray, resolution=100):
        """
        Converts a grayscale image into a Cubical Complex suitable for persistent homology computation.
        
        Args:
            image_gray (np.ndarray): Grayscale image as a NumPy array.
            resolution (int): Target resolution for resizing the image.
        
        Returns:
            gd.CubicalComplex: Cubical complex representation of the image.
        """
        # Resize image to a standard resolution for consistent analysis
        image_resized = cv2.resize(image_gray, (resolution, resolution))
        
        # Convert image to float32 array
        image_array = np.array(image_resized, dtype=np.float32)
        
        # Pad edges to avoid boundary effects during complex creation
        vertex_array = np.pad(image_array, pad_width=1, mode='edge')
        
        # Create and return a Cubical Complex object from the pixel values
        return gd.CubicalComplex(vertices=vertex_array)

    # --- Helper function: Compute persistent homology for a single image ---
    def compute_image_persistence(image_path):
        """
        Reads an image, converts it to a Cubical Complex, and computes persistence.
        
        Args:
            image_path (str): Full path to the grayscale image file.
        
        Returns:
            list or None: Persistence diagram, or None if image cannot be loaded.
        """
        # Read image in grayscale mode
        image_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image_gray is None:
            print(f"⚠️ Error: Could not load image {image_path}")
            return None
        
        # Create cubical complex and compute persistence
        cubical_complex = image_to_cubical_complex(image_gray)
        return cubical_complex.persistence()

    # --- Helper function: Process all images inside a single subfolder ---
    def process_folder(folder_name, folder_path):
        """
        Process a single subfolder containing images and save persistence results.
        
        Args:
            folder_name (str): Name of the subfolder being processed.
            folder_path (str): Full path to that subfolder.
        
        Returns:
            dict: Persistence results for images in this folder.
        """
        # Collect all valid image files
        image_files = [
            f for f in os.listdir(folder_path)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]
        
        # Skip folder if no images found
        if not image_files:
            print(f"⚠️ No images found in {folder_path}")
            return {}

        # Create an output directory for persistence results
        folder_output_dir = os.path.join(persistence_results_dir, folder_name)
        os.makedirs(folder_output_dir, exist_ok=True)

        persistence_results = {}
        
        # Process each image file individually
        for filename in image_files:
            image_path = os.path.join(folder_path, filename)
            persistence = compute_image_persistence(image_path)
            
            # Save results if successfully computed
            if persistence:
                persistence_results[filename] = persistence
                
                # Save persistence data as a pickle file
                output_file = os.path.join(folder_output_dir, f"{filename}_persistence.pkl")
                with open(output_file, "wb") as f:
                    pickle.dump(persistence, f)
        
        print(f"✅ Saved persistence results for '{folder_name}' in '{folder_output_dir}'")
        return persistence_results

    # --- Main Execution: Sequentially process all subfolders ---
    all_persistence = {}
    for subfolder in subfolders:
        folder_name = os.path.basename(subfolder)
        result = process_folder(folder_name, subfolder)
        if result:
            all_persistence[folder_name] = result

    return all_persistence


In [9]:
import pickle
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()

def load_persistence_diagram(file_path):
    """
    Load a persistence diagram from a pickle file and return H0 and H1 components as NumPy arrays.
    Handles various data formats typically saved in persistence computation.
    """

    try:
        # Step 1: Load data from the pickle file
        with open(file_path, 'rb') as f:
            data = pickle.load(f)

        h0, h1 = [], []  # Initialize lists for H0 and H1

        # Format: list of (dim, (birth, death)) tuples
        if isinstance(data, list) and all(isinstance(d, tuple) and isinstance(d[1], tuple) for d in data):
            for dim, (birth, death) in data:
                if dim == 0:
                    h0.append([birth, death])
                elif dim == 1:
                    h1.append([birth, death])
            return np.array(h0), np.array(h1)

        # Format: list of two arrays/lists: [H0, H1]
        elif isinstance(data, list) and len(data) == 2:
            return np.array(data[0]), np.array(data[1])

        # Format: dictionary with 'H0' and 'H1' keys
        elif isinstance(data, dict):
            h0 = data.get("H0", [])
            h1 = data.get("H1", [])
            return np.array(h0), np.array(h1)

        else:
            print(f"⚠️ Unexpected format in {file_path}")
            return np.array([]), np.array([])

    except Exception as e:
        print(f"❌ Failed to load {file_path}: {e}")
        return np.array([]), np.array([])


## Calculating Bottleneck and Wasserstein Distance

In [11]:
def compute_bottleneck_distances(folder_path):
    """
    Compute pairwise bottleneck distances (H0 and H1) for all persistence diagrams in a folder.
    Works well in Jupyter Notebook.
    """
    folder_path = Path(folder_path)  # Ensure it's a Path object
    folder_name = folder_path.name
    print(f"\n📁 Processing folder: {folder_name}")

    # Step 1: Get all .pkl files
    ph_files = sorted(folder_path.glob("*.pkl"))

    if len(ph_files) < 2:
        print("⚠️ Not enough files to compute distances.")
        return None, None

    # Step 2: Load all diagrams
    diagrams = {}
    for file in tqdm(ph_files, desc="📦 Loading diagrams", disable=True):
        h0, h1 = load_persistence_diagram(file)
        if h0 is not None and h1 is not None:
            diagrams[file.name] = (h0, h1)

    if len(diagrams) < 2:
        print("⚠️ Not enough valid diagrams.")
        return None, None

    # Step 3: Prepare distance matrices
    files = list(diagrams.keys())
    n = len(files)
    dist_h0 = np.zeros((n, n))
    dist_h1 = np.zeros((n, n))

    # Step 4: Compute distances
    for i, j in tqdm(itertools.combinations(range(n), 2), total=n*(n-1)//2, desc="🔁 Computing distances", disable=True):
        f1, f2 = files[i], files[j]
        try:
            d0 = gd.bottleneck_distance(diagrams[f1][0], diagrams[f2][0])
            d1 = gd.bottleneck_distance(diagrams[f1][1], diagrams[f2][1])

            dist_h0[i, j] = dist_h0[j, i] = d0
            dist_h1[i, j] = dist_h1[j, i] = d1

        except Exception as e:
            print(f"❌ Error between {f1} and {f2}: {e}")
            dist_h0[i, j] = dist_h0[j, i] = np.nan
            dist_h1[i, j] = dist_h1[j, i] = np.nan

    # Step 5: Convert to DataFrames
    df_h0 = pd.DataFrame(dist_h0, index=files, columns=files)
    df_h1 = pd.DataFrame(dist_h1, index=files, columns=files)

    print("✅ Bottleneck distance computation complete!")
    return df_h0, df_h1


In [12]:
def compute_wasserstein_distances(folder_path, order=1):
    """
    Compute pairwise Wasserstein distances (H0 and H1) between persistence diagrams in a folder.
    Works smoothly in Jupyter Notebook.
    """
    folder_path = Path(folder_path)
    folder_name = folder_path.name
    print(f"\n📂 Processing: {folder_name}")

    # Step 1: Find all .pkl files
    pkl_files = sorted(folder_path.glob("*.pkl"))
    if len(pkl_files) < 2:
        print("⚠️ Not enough files to compute distances.")
        return None, None

    # Step 2: Load persistence diagrams
    diagrams = {}
    for file in tqdm(pkl_files, desc="📦 Loading diagrams", disable=True):
        h0, h1 = load_persistence_diagram(file)
        if h0 is not None and h1 is not None:
            diagrams[file.name] = (h0, h1)

    if len(diagrams) < 2:
        print("⚠️ Not enough valid diagrams.")
        return None, None

    # Step 3: Prepare distance matrices
    image_names = list(diagrams.keys())
    n = len(image_names)
    wasserstein_h0_matrix = np.zeros((n, n))
    wasserstein_h1_matrix = np.zeros((n, n))

    # Step 4: Compute distances with progress
    for i, j in tqdm(itertools.combinations(range(n), 2), total=n*(n-1)//2, desc="🔁 Computing distances", disable=True):
        f1, f2 = image_names[i], image_names[j]
        try:
            d0 = gw.wasserstein_distance(diagrams[f1][0], diagrams[f2][0], order=order)
            d1 = gw.wasserstein_distance(diagrams[f1][1], diagrams[f2][1], order=order)

            wasserstein_h0_matrix[i, j] = wasserstein_h0_matrix[j, i] = d0
            wasserstein_h1_matrix[i, j] = wasserstein_h1_matrix[j, i] = d1
        except Exception as e:
            print(f"❌ Error between {f1} and {f2}: {e}")
            wasserstein_h0_matrix[i, j] = wasserstein_h0_matrix[j, i] = np.nan
            wasserstein_h1_matrix[i, j] = wasserstein_h1_matrix[j, i] = np.nan

    # Step 5: Create DataFrames
    df_h0 = pd.DataFrame(wasserstein_h0_matrix, index=image_names, columns=image_names)
    df_h1 = pd.DataFrame(wasserstein_h1_matrix, index=image_names, columns=image_names)

    print("✅ Wasserstein distance computation complete!")
    return df_h0, df_h1


## Permutation Test Calculation

In [14]:
def perform_permutation_test(artist_name, distance_df, num_permutations=10000, show_progress=True):
    """
    Perform a permutation test to assess whether the average distance between
    one artist's images and other images is significantly different from random.

    Args:
        artist_name (str): Name/prefix used to identify the artist's images.
        distance_df (pd.DataFrame): Pairwise distance matrix (DataFrame).
        num_permutations (int): Number of permutations for generating null distribution.
        show_progress (bool): Whether to show progress bar.

    Returns:
        p_value (float): The probability of observing the mean distance or lower by chance.
        observed_mean (float): The actual mean distance between the artist and others.
    """
    # Clean index: remove non-printable characters and strip whitespace
    distance_df.index = distance_df.index.to_series().apply(
        lambda x: ''.join(filter(str.isprintable, str(x)))
    ).str.strip()
    distance_df.columns = distance_df.columns.to_series().apply(
        lambda x: ''.join(filter(str.isprintable, str(x)))
    ).str.strip()

    #Matrix with all distances
    all_distances = distance_df.values[np.triu_indices_from(distance_df.values, k=1)]

    # Identify the artist's images and the others' images
    artist_images = [img for img in distance_df.index if img.startswith(artist_name)]
    other_images = [img for img in distance_df.index if not img.startswith(artist_name)]
    # sanity check
    if not artist_images or not other_images:
        raise ValueError(f"No valid images for artist '{artist_name}'.")

    # Compute observed mean distance
    observed_distances = [distance_df.loc[img1, img2] for img1 in artist_images for img2 in other_images if img2 in distance_df.columns]
    # sanity check
    if not observed_distances:
        raise ValueError(f"No valid distances found for artist '{artist_name}'.")

    observed_mean = np.mean(observed_distances)

    permutation_means = [observed_mean]
    
    all_divisions = []
    i = 0
    for combo1 in itertools.combinations(distance_df.columns, len(artist_images)):
        list1 = list(combo1)
        list2 = [item for item in distance_df if item not in list1]
        all_divisions.append((list1, list2))
        i += 1
        if i == num_permutations:
            break
        
    for (sample1,sample2) in all_divisions:
        shuffled_distances = [distance_df.loc[img1, img2] for img1 in sample1 for img2 in sample2 if img2 in distance_df.columns]
        permutation_means.append(np.mean(shuffled_distances))

    # Compute p-value
    if observed_mean == min(permutation_means):
        print(f" p-value was min for {artist_name}")
        p_value = np.mean(np.array(permutation_means) <= observed_mean)
    else:
        p_value = np.mean(np.array(permutation_means) < observed_mean)
        if observed_mean == max(permutation_means):
            print(f" p-value was max for {artist_name}")

    # Print results
    print(f"\n🎨 Artist: {artist_name}")
    print(f"🔍 Observed Mean Distance: {observed_mean:.4f}")
    print(f"📊 p-value: {p_value:.4f} (based on {num_permutations} permutations)")

    return p_value, observed_mean

In [15]:
def analyze_distances_for_file(file_path, distance_type="Bottleneck", output_folder=None):
    """
    Analyze a pairwise distance matrix (CSV) and perform permutation tests per artist.

    Args:
        file_path (str or Path): Path to the CSV file containing the distance matrix.
        distance_type (str): Type of distance ('Bottleneck' or 'Wasserstein') for labeling results.
        output_folder (str or Path, optional): Folder to save the results.

    Returns:
        results_df (pd.DataFrame): DataFrame containing p-values and significance results per artist.
    """
    try:
        file_path = Path(file_path)  # Convert to Path if it's a string
        df = pd.read_csv(file_path, index_col=0)

        # Clean index and column names
        df.index = df.index.to_series().apply(
            lambda x: ''.join(filter(str.isprintable, str(x)))
        ).str.strip()
        df.columns = df.columns.to_series().apply(
            lambda x: ''.join(filter(str.isprintable, str(x)))
        ).str.strip()

        print(f"\n📄 Processing file: {file_path.name}")

        # Extract unique artist names
        artists = sorted(set(img.split('.')[0].rstrip('0123456789') for img in df.index))
        print(f"🎨 Detected artists: {artists}")

        results = []

        # Run permutation test per artist
        for artist in artists:
            try:
                p_value, mean_distance = perform_permutation_test(artist, df)
                results.append({
                    'Artist': artist,
                    'P-Value': f"{p_value:.4f}",
                    'Mean_Distance': f"{mean_distance:.4f}",
                    'Significance': 'Significant' if p_value < 0.025 or p_value > 0.975 else 'Not Significant'
                })
            except (KeyError, ValueError) as e:
                print(f"⚠️ Error for artist '{artist}' in file '{file_path.name}': {e}")
                continue

        results_df = pd.DataFrame(results)

        # Set output folder
        if output_folder is None:
            output_folder = file_path.parent
        else:
            output_folder = Path(output_folder)
            output_folder.mkdir(parents=True, exist_ok=True)  # Create if doesn't exist

        # Save results
        output_file = output_folder / (file_path.stem + f'_results_{distance_type}.csv')
        results_df.to_csv(output_file, index=False)
        print(f"✅ Results saved to: {output_file}")

        return results_df

    except Exception as e:
        print(f"❌ Error processing file '{file_path}': {e}")
        return pd.DataFrame()


In [16]:
def analyze_distances_in_folder(folder_path, distance_type="Bottleneck"):
    """
    Analyze all CSV distance matrices in a folder using permutation tests.

    Args:
        folder_path (str or Path): Path to the folder containing CSV distance matrices.
        distance_type (str): 'Bottleneck' or 'Wasserstein' for labeling result files.

    Returns:
        all_results (dict): A dictionary mapping each CSV filename to its result DataFrame.
    """
    folder_path = Path(folder_path)  # Ensure it's a Path object

    if not folder_path.exists() or not folder_path.is_dir():
        print(f"❌ Folder not found: {folder_path}")
        return {}

    # Define output folder
    output_folder = folder_path.parent / f"{distance_type}_PTest"
    output_folder.mkdir(parents=True, exist_ok=True)  # Create if doesn't exist

    # Gather all CSV files in the folder
    csv_files = list(folder_path.glob("*.csv"))

    if not csv_files:
        print(f"⚠️ No CSV files found in {folder_path}")
        return {}

    all_results = {}

    # Process each CSV file
    for csv_file in csv_files:
        print(f"\n📄 Processing file: {csv_file.name}")
        results_df = analyze_distances_for_file(
            csv_file, distance_type=distance_type, output_folder=output_folder
        )
        all_results[csv_file.name] = results_df

    print(f"\n✅ Completed analysis of {len(all_results)} files.")
    return all_results


## Experiments set up

In [18]:
# --- Extract artist from filename ---
def extract_artist(filename):
    base = os.path.splitext(filename)[0]
    if base.endswith('_persistence'):
        base = base[:-len('_persistence')]
    for ext in ['.jpg', '.jpeg', '.png']:
        if base.endswith(ext):
            base = base[:-len(ext)]
            break
    parts = base.split('_')
    if len(parts) > 1 and len(parts[0]) == 1:
        artist_part = parts[1]
    else:
        artist_part = parts[0]
    artist_name = ''.join(filter(str.isalpha, artist_part))
    return artist_name

In [19]:
def process_all_folders(base_dir, output_dir, categories, method="bottleneck", order=1):
    """
    Compute Bottleneck or Wasserstein distances for all subfolders inside a base directory.
    Each subfolder must contain .pkl files with persistence diagrams.

    Args:
        base_dir (str or Path): Folder containing subfolders of diagrams.
        output_dir (str or Path): Folder to save the resulting distance CSVs.
        method (str): Either 'bottleneck' or 'wasserstein'.
        order (int): Order for Wasserstein distance (only used if method='wasserstein').
    """
    base_dir = Path(base_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    subfolders = [f for f in base_dir.iterdir() if f.is_dir()]
    if not subfolders:
        print("⚠️ No subfolders found to process.")
        return

    print(f"\n📦 Found {len(subfolders)} folders in '{base_dir.name}'")

    for folder in tqdm(subfolders, desc="🔁 Processing folders"):
        print(f"\n📁 Processing: {folder.name} ({method} distance)")

        if(folder.name not in categories):
            continue

        # Compute distances using the selected method
        if method == "bottleneck":
            df_h0, df_h1 = compute_bottleneck_distances(folder)
            suffix = "bottleneck"
        elif method == "wasserstein":
            df_h0, df_h1 = compute_wasserstein_distances(folder, order=order)
            suffix = f"wasserstein_order{order}"
        else:
            print(f"❌ Unsupported method: {method}")
            continue

        # Save outputs if both distance matrices are valid
        if df_h0 is not None and df_h1 is not None:
            df_h0_path = output_dir / f"{folder.name}_H0_{suffix}.csv"
            df_h1_path = output_dir / f"{folder.name}_H1_{suffix}.csv"
            df_h0.to_csv(df_h0_path)
            df_h1.to_csv(df_h1_path)
            print(f"✅ Saved: {df_h0_path.name}, {df_h1_path.name}")
            #print(f"✅ Saved: {path.name}")
        else:
            print(f"⚠️ Skipped {folder.name} (invalid/missing data)")


In [20]:
def compute_avg_dist(bottleneck_distances_dir, output_dir):
    """
    Compute the average bottleneck distance from a set of CSV files.

    Each CSV file in the specified directory is expected to contain
    a numeric matrix of pairwise distances (with labels in the first
    row and column). This function computes the mean value of all
    numeric entries in each CSV file and writes a summary CSV.

    Args:
        bottleneck_distances_dir (str): Path to the directory containing CSV files.
        output_dir (str): Directory where the summary CSV file will be saved.

    Returns:
        None
    """
    
    # --- Step 1: Get list of all CSV files in the bottleneck distances directory ---
    csv_files = [
        f for f in os.listdir(bottleneck_distances_dir)
        if f.endswith(".csv")
    ]

    results = []  # To store average values for each file

    # --- Step 2: Process each CSV file one by one ---
    for file in csv_files:
        file_path = os.path.join(bottleneck_distances_dir, file)

        # Read the CSV file into a pandas DataFrame
        # The first column and row are assumed to contain labels (so use index_col=0)
        df = pd.read_csv(file_path, index_col=0)

        # Convert all entries to numeric (ignore non-numeric data safely)
        df = df.apply(pd.to_numeric, errors="coerce")

        # Compute the average (mean) of all numeric entries in the matrix
        avg_value = df.values.mean()

        # Store the filename and computed average in the results list
        results.append({
            "file": file,
            "average": avg_value
        })

    # --- Step 3: Save all computed averages into a single summary CSV file ---
    summary_path = os.path.join(output_dir, "averages.csv")
    pd.DataFrame(results).to_csv(summary_path, index=False)

    # --- Step 4: Log success message ---
    print(f"✅ Per-file averages saved to: {summary_path}")


In [21]:
def process_artist_analysis(base_dir, image_dir, output_folder):
    """
    Main pipeline to process images, compute persistent homology, 
    and analyze bottleneck distances for different image types.

    This function performs the following steps:
      1. Preprocesses the images (grayscale, edges, and RGB).
      2. Computes persistent homology for processed images.
      3. Calculates bottleneck distances between persistence diagrams.
      4. Computes the average bottleneck distance for summary analysis.

    Args:
        base_dir (str): Root directory for storing intermediate and final results.
        image_dir (str): Directory containing original input images.

    Returns:
        None
    """

    # --- Step 1: Define output folder paths for each processing stage ---
    processed_images_dir = os.path.join(base_dir, "processed_images")
    persistence_results_dir = os.path.join(base_dir, "persistence_results")
    bottleneck_distance_dir = os.path.join(output_folder, "bottleneck_distances")
    avg_distance_dir = os.path.join(output_folder, "avg_dist")

    # --- Step 2: Create directories if they do not already exist ---
    os.makedirs(processed_images_dir, exist_ok=True)
    os.makedirs(persistence_results_dir, exist_ok=True)
    os.makedirs(bottleneck_distance_dir, exist_ok=True)
    os.makedirs(avg_distance_dir, exist_ok=True)

    # --- Step 3: Process and save different image variants ---
    # Converts input images into grayscale, edge-detected, and RGB variations.
    process_images(image_dir, processed_images_dir)

    # --- Step 4: Compute persistent homology for processed images ---
    # This step generates persistence diagrams for each processed image.
    compute_persistent_homology(processed_images_dir, persistence_results_dir)


def process_images(image_dir, output_dir):
    """
    Preprocess input images into different representations
    for persistent homology analysis.

    This function generates:
      - Grayscale channel
      - Edge-detected channel
      - RGB component-separated channels

    Args:
        image_dir (str): Directory containing original images.
        output_dir (str): Directory to save processed images.

    Returns:
        None
    """
    
    # --- Step 1: Process grayscale images ---
    process_grey_images(image_dir, output_dir)

    # --- Step 2: Process edge-detected images ---
    process_edge_images(image_dir, output_dir)

    # --- Step 3: Process RGB-based images ---
    process_RGB_images(image_dir, output_dir)


In [22]:
def predict_analysis(base_dir, image_dir, output_folder):
    persistence_results_dir = os.path.join(base_dir, "persistence_results")
    bottleneck_distance_dir = os.path.join(output_folder, "bottleneck_distances")
    wasserstein_distance_dir = os.path.join(output_folder, "wasserstein_distances")
    avg_distance_dir = os.path.join(output_folder, "avg_dist")
    
    # --- Step 4: Compute bottleneck distances between persistence diagrams ---
    categories = ["red_channel", "blue_channel", "green_channel", "grayscale_channel", "edge_channel"] #May choose to compute only fewer channels
    process_all_folders(persistence_results_dir, bottleneck_distance_dir, categories, method="bottleneck")

    # --- Step 5: Compute wasserstein distances between persistence diagrams ---
    # This measures topological similarity between pairs of images.
    categories = ["red_channel", "blue_channel", "green_channel", "grayscale_channel", "edge_channel"] #May choose to compute only fewer channels
    process_all_folders(persistence_results_dir, wasserstein_distance_dir, categories, method="wasserstein")

    # --- Step 6: Compute average bottleneck distance from all results ---
    # Summarizes the distances into a single CSV report.
    compute_avg_dist(bottleneck_distance_dir, avg_distance_dir)

    print("✅ Artist analysis pipeline completed successfully.")
    

## Running experiments 

In [24]:
### Impressionism ###

#
#Split each image in the (sub)folder(s) into its 5 channels
#
test_folder = "paintings/Impressionism/"
output_folder = "paintings/Impressionism/analysis"

folders = [name for name in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, name))]

for artist in folders:
    base_dir = os.path.join(output_folder)
    image_dir = os.path.join(test_folder,artist)
    process_artist_analysis(base_dir, image_dir, output_folder)

predict_analysis(base_dir, image_dir, output_folder)

#
#Bottleneck
#
folder_path = "paintings/Impressionism/analysis/bottleneck_distances"  # Define the folder path and distance type
distance_type = "Bottleneck"

# Call the function to analyze distances
all_results = analyze_distances_in_folder(folder_path, distance_type=distance_type)

# Display the results
for file_name, result_df in all_results.items():
    print(f"\nResults for {file_name}:")
    print("=" * 80)
    print(result_df.to_string(index=False))

# 
# Wasserstein
#
folder_path = "paintings/Impressionism/analysis/wasserstein_distances"  
distance_type = "Wasserstein"

# Call the function to analyze distances
all_results = analyze_distances_in_folder(folder_path, distance_type=distance_type)

# Display the results
for file_name, result_df in all_results.items():
    print(f"\nResults for {file_name}:")
    print("=" * 80)
    print(result_df.to_string(index=False))


Processed 10 images and saved them in grayscale folder.
Processed 10 images and saved them in the edge-detection folder.
Processed 10 images and saved them in RGB channel folders.
✅ Saved persistence results for 'blue_channel' in 'paintings/Impressionism/analysis\persistence_results\blue_channel'
✅ Saved persistence results for 'edge_channel' in 'paintings/Impressionism/analysis\persistence_results\edge_channel'
✅ Saved persistence results for 'grayscale_channel' in 'paintings/Impressionism/analysis\persistence_results\grayscale_channel'
✅ Saved persistence results for 'green_channel' in 'paintings/Impressionism/analysis\persistence_results\green_channel'
✅ Saved persistence results for 'red_channel' in 'paintings/Impressionism/analysis\persistence_results\red_channel'
Processed 10 images and saved them in grayscale folder.
Processed 10 images and saved them in the edge-detection folder.
Processed 10 images and saved them in RGB channel folders.
✅ Saved persistence results for 'blue_ch

🔁 Processing folders:   0%|          | 0/5 [00:00<?, ?it/s]


📁 Processing: blue_channel (bottleneck distance)

📁 Processing folder: blue_channel
✅ Bottleneck distance computation complete!
✅ Saved: blue_channel_H0_bottleneck.csv, blue_channel_H1_bottleneck.csv

📁 Processing: edge_channel (bottleneck distance)

📁 Processing folder: edge_channel
✅ Bottleneck distance computation complete!
✅ Saved: edge_channel_H0_bottleneck.csv, edge_channel_H1_bottleneck.csv

📁 Processing: grayscale_channel (bottleneck distance)

📁 Processing folder: grayscale_channel
✅ Bottleneck distance computation complete!
✅ Saved: grayscale_channel_H0_bottleneck.csv, grayscale_channel_H1_bottleneck.csv

📁 Processing: green_channel (bottleneck distance)

📁 Processing folder: green_channel
✅ Bottleneck distance computation complete!
✅ Saved: green_channel_H0_bottleneck.csv, green_channel_H1_bottleneck.csv

📁 Processing: red_channel (bottleneck distance)

📁 Processing folder: red_channel
✅ Bottleneck distance computation complete!
✅ Saved: red_channel_H0_bottleneck.csv, red_c

🔁 Processing folders:   0%|          | 0/5 [00:00<?, ?it/s]


📁 Processing: blue_channel (wasserstein distance)

📂 Processing: blue_channel
✅ Wasserstein distance computation complete!
✅ Saved: blue_channel_H0_wasserstein_order1.csv, blue_channel_H1_wasserstein_order1.csv

📁 Processing: edge_channel (wasserstein distance)

📂 Processing: edge_channel
✅ Wasserstein distance computation complete!
✅ Saved: edge_channel_H0_wasserstein_order1.csv, edge_channel_H1_wasserstein_order1.csv

📁 Processing: grayscale_channel (wasserstein distance)

📂 Processing: grayscale_channel
✅ Wasserstein distance computation complete!
✅ Saved: grayscale_channel_H0_wasserstein_order1.csv, grayscale_channel_H1_wasserstein_order1.csv

📁 Processing: green_channel (wasserstein distance)

📂 Processing: green_channel
✅ Wasserstein distance computation complete!
✅ Saved: green_channel_H0_wasserstein_order1.csv, green_channel_H1_wasserstein_order1.csv

📁 Processing: red_channel (wasserstein distance)

📂 Processing: red_channel
✅ Wasserstein distance computation complete!
✅ Save

In [25]:
### AcademicRealism analysis ###

#
# Split each image in the (sub)folder(s) into its 5 channels
#
test_folder = "paintings/AcademicRealism/"
output_folder = "paintings/AcademicRealism/analysis"

folders = [name for name in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, name))]

for artist in folders:
    base_dir = os.path.join(output_folder)
    image_dir = os.path.join(test_folder,artist)
    process_artist_analysis(base_dir, image_dir, output_folder)

predict_analysis(base_dir, image_dir, output_folder)

#
# Bottleneck
#
folder_path = "paintings/AcademicRealism/analysis/bottleneck_distances"  
distance_type = "Bottleneck"

# Call the function to analyze distances
all_results = analyze_distances_in_folder(folder_path, distance_type=distance_type)

# Display the results
for file_name, result_df in all_results.items():
    print(f"\nResults for {file_name}:")
    print("=" * 80)
    print(result_df.to_string(index=False))
#
# Wasserstein
#
folder_path = "paintings/AcademicRealism/analysis/wasserstein_distances"  
distance_type = "Wasserstein"

# Call the function to analyze distances
all_results = analyze_distances_in_folder(folder_path, distance_type=distance_type)

# Display the results
for file_name, result_df in all_results.items():
    print(f"\nResults for {file_name}:")
    print("=" * 80)
    print(result_df.to_string(index=False))



Processed 10 images and saved them in grayscale folder.
Processed 10 images and saved them in the edge-detection folder.
Processed 10 images and saved them in RGB channel folders.
✅ Saved persistence results for 'blue_channel' in 'paintings/AcademicRealism/analysis\persistence_results\blue_channel'
✅ Saved persistence results for 'edge_channel' in 'paintings/AcademicRealism/analysis\persistence_results\edge_channel'
✅ Saved persistence results for 'grayscale_channel' in 'paintings/AcademicRealism/analysis\persistence_results\grayscale_channel'
✅ Saved persistence results for 'green_channel' in 'paintings/AcademicRealism/analysis\persistence_results\green_channel'
✅ Saved persistence results for 'red_channel' in 'paintings/AcademicRealism/analysis\persistence_results\red_channel'
Processed 10 images and saved them in grayscale folder.
Processed 10 images and saved them in the edge-detection folder.
Processed 10 images and saved them in RGB channel folders.
✅ Saved persistence results fo

🔁 Processing folders:   0%|          | 0/5 [00:00<?, ?it/s]


📁 Processing: blue_channel (bottleneck distance)

📁 Processing folder: blue_channel
✅ Bottleneck distance computation complete!
✅ Saved: blue_channel_H0_bottleneck.csv, blue_channel_H1_bottleneck.csv

📁 Processing: edge_channel (bottleneck distance)

📁 Processing folder: edge_channel
✅ Bottleneck distance computation complete!
✅ Saved: edge_channel_H0_bottleneck.csv, edge_channel_H1_bottleneck.csv

📁 Processing: grayscale_channel (bottleneck distance)

📁 Processing folder: grayscale_channel
✅ Bottleneck distance computation complete!
✅ Saved: grayscale_channel_H0_bottleneck.csv, grayscale_channel_H1_bottleneck.csv

📁 Processing: green_channel (bottleneck distance)

📁 Processing folder: green_channel
✅ Bottleneck distance computation complete!
✅ Saved: green_channel_H0_bottleneck.csv, green_channel_H1_bottleneck.csv

📁 Processing: red_channel (bottleneck distance)

📁 Processing folder: red_channel
✅ Bottleneck distance computation complete!
✅ Saved: red_channel_H0_bottleneck.csv, red_c

🔁 Processing folders:   0%|          | 0/5 [00:00<?, ?it/s]


📁 Processing: blue_channel (wasserstein distance)

📂 Processing: blue_channel
✅ Wasserstein distance computation complete!
✅ Saved: blue_channel_H0_wasserstein_order1.csv, blue_channel_H1_wasserstein_order1.csv

📁 Processing: edge_channel (wasserstein distance)

📂 Processing: edge_channel


C:\Users\bgiun\anaconda3\Lib\site-packages\ot\lp\__init__.py:557: RuntimeWarning: invalid value encountered in divide
  b = b * a.sum(0) / b.sum(0, keepdims=True)
C:\Users\bgiun\anaconda3\Lib\site-packages\ot\lp\__init__.py:630: UserWarning: Problem infeasible. Check that a and b are in the simplex
  check_result(result_code)


✅ Wasserstein distance computation complete!
✅ Saved: edge_channel_H0_wasserstein_order1.csv, edge_channel_H1_wasserstein_order1.csv

📁 Processing: grayscale_channel (wasserstein distance)

📂 Processing: grayscale_channel
✅ Wasserstein distance computation complete!
✅ Saved: grayscale_channel_H0_wasserstein_order1.csv, grayscale_channel_H1_wasserstein_order1.csv

📁 Processing: green_channel (wasserstein distance)

📂 Processing: green_channel
✅ Wasserstein distance computation complete!
✅ Saved: green_channel_H0_wasserstein_order1.csv, green_channel_H1_wasserstein_order1.csv

📁 Processing: red_channel (wasserstein distance)

📂 Processing: red_channel
✅ Wasserstein distance computation complete!
✅ Saved: red_channel_H0_wasserstein_order1.csv, red_channel_H1_wasserstein_order1.csv
✅ Per-file averages saved to: paintings/AcademicRealism/analysis\avg_dist\averages.csv
✅ Artist analysis pipeline completed successfully.

📄 Processing file: blue_channel_H0_bottleneck.csv

📄 Processing file: bl